In [1]:
from pathlib import Path
from uuid import UUID, uuid4

from pydantic_ai import Agent, RunContext

from knd.memory import AgentMemories

%load_ext autoreload
%autoreload 2

In [2]:
SUMMARY_LIMIT = 20_000
MESSAGE_COUNT_LIMIT = 20
MEMORIES_DIR = Path("memories")

In [3]:
user_id = uuid4()
user_id = UUID("db5fe6ca-55ae-4f38-9acf-62d707a46041")

In [4]:
agent_name = "anime_fan"

anime_agent = Agent(
    name=agent_name,
    model="google-gla:gemini-1.5-flash",
    system_prompt="You are an anime fan. The user will also be an anime fan. Just bros chilling talking about anime.",
    deps_type=AgentMemories,
    result_type=str,
)


@anime_agent.system_prompt(dynamic=True)
def system_prompt(ctx: RunContext[AgentMemories]) -> str:
    return str(ctx.deps)


memory_agent = Agent(name="memory_agent", model="google-gla:gemini-1.5-flash")


deps = AgentMemories.load(
    agent_name=agent_name, user_id=user_id, memories_dir=MEMORIES_DIR, message_count_limit=MESSAGE_COUNT_LIMIT
)
message_history = deps.message_history

In [5]:
deps.model_dump()

{'user_specific_experience': {'profile': {'name': '',
   'age': None,
   'interests': ['action anime', 'Solo Leveling', 'Cowboy Bebop'],
   'home': '',
   'occupation': '',
   'conversation_preferences': []},
  'memories': [{'id': 'ad1f52c9-a1fd-4276-8eee-eb547023e086',
    'created_at': '2025-01-17 18:47:01',
    'context': 'This preference should be used to recommend similar anime in future conversations.  Useful when the user mentions wanting recommendations or discussing action anime.',
    'category': 'preference',
    'content': 'Enjoys action anime with overpowered protagonists, like Solo Leveling.',
    'superseded_ids': []},
   {'id': '8862a227-45c1-409a-a0d8-41ddf966236b',
    'created_at': '2025-01-17 18:47:01',
    'context': 'This preference can be used to suggest anime with high-quality animation and action sequences.  Useful for recommendations, especially if the user specifies a preference for visually appealing anime.',
    'category': 'preference',
    'content': 'Lik

In [6]:
user_prompt = "guess which one I watched"

while user_prompt.lower() not in ["q", "quit", "exit"]:
    res = await anime_agent.run(user_prompt=user_prompt, deps=deps, message_history=message_history)
    message_history = res.all_messages()
    user_prompt = input(f"{res.data}    (q to quit)> ")

In [ ]:
message_history